In [1]:
!cd /content

In [2]:
!pip install git+https://github.com/facebookresearch/segment-anything.git



  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-0nbxe64z
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-0nbxe64z
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=f3508f6be8474557ea6dce586c8006c64477dde2e93b98dd862d43ad70985f0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1p3ni92s/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [14]:
#=========================================================
#================== Create Annotation ====================
#=========================================================


import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from segment_anything import sam_model_registry, SamPredictor
import json
import datetime

# Define paths
path = "train_test"
image_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/' + path
mask_save_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/masked_images_1/' + path
annotations_save_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/test'

# Ensure the mask save directory exists
os.makedirs(mask_save_dir, exist_ok=True)
os.makedirs(annotations_save_dir, exist_ok=True)

# Load the SAM model
sam_checkpoint = "/content/drive/MyDrive/plant_disease_detection/models/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# Initialize COCO Panoptic annotation format
coco_output = {
    "info": {
        "description": "Plant Disease Detection Dataset",
        "version": "1.0",
        "year": datetime.datetime.now().year,
        "contributor": "Labeba Tahsin",
        "date_created": datetime.datetime.now().isoformat()
    },
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}

category_id = 1  # Assuming one category for simplicity

# Add category information (assuming one category for this example)
coco_output["categories"].append({
    "id": category_id,
    "name": "plant_disease",
    "supercategory": "plant"
})

annotation_id = 1

# Iterate over all subfolders and images in the directory
for root, dirs, files in os.walk(image_dir):
    for image_file in files:
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            try:
                # Load the image
                image_path = os.path.join(root, image_file)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Failed to load image {image_file}")
                    continue
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                height, width, _ = image_rgb.shape

                # Update file name with full path for COCO annotation
                relative_path = os.path.relpath(root, image_dir)
                full_image_path = os.path.join('/content/drive/MyDrive/plant_disease_detection/dataset', path, relative_path, image_file)

                # Add image info to COCO
                image_info = {
                    "id": annotation_id,
                    "file_name": full_image_path,
                    "height": height,
                    "width": width
                }
                coco_output["images"].append(image_info)

                # Set the image for prediction
                predictor.set_image(image_rgb)

                # Define central point
                central_point = np.array([width // 2, height // 2])
                input_points = np.vstack([central_point])
                input_labels = np.ones(input_points.shape[0])
                input_box = np.array([8, 8, width - 8, height - 8])

                # Predict masks using the input points
                masks, scores, logits = predictor.predict(
                    point_coords=input_points,
                    point_labels=input_labels,
                    box=input_box,
                    multimask_output=True,
                )

                # Identify the mask with the highest score
                best_mask_idx = np.argmax(scores)
                best_mask = masks[best_mask_idx]
                best_score = scores[best_mask_idx]

                # Draw the mask and box on the overlay
                overlay = image_rgb.copy()
                color = np.array([255 / 255, 20 / 255, 147 / 255, 0.6])  # Pinkish color
                h, w = best_mask.shape[-2:]
                mask_image = best_mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
                overlay = cv2.addWeighted(overlay, 0.7, (mask_image[:, :, :3] * 255).astype(np.uint8), 0.3, 0)

                # Draw the box on the overlay
                x0, y0, x1, y1 = input_box
                cv2.rectangle(overlay, (x0, y0), (x1, y1), (0, 255, 0), 1)  # Lawngreen color

                # Create corresponding subfolder in the mask save directory
                save_subdir = os.path.join(mask_save_dir, relative_path)
                os.makedirs(save_subdir, exist_ok=True)

                # Save the overlay image
                overlay_path = os.path.join(save_subdir, f"overlay_{image_file}")
                cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

                # Prepare annotation for COCO
                segmentation = np.zeros((height, width), dtype=np.uint8)
                segmentation[best_mask] = annotation_id

                annotation_info = {
                    "id": annotation_id,
                    "image_id": annotation_id,
                    "category_id": category_id,
                    "segmentation": segmentation.tolist(),
                    "area": int(np.sum(best_mask)),
                    "bbox": [int(x0), int(y0), int(x1 - x0), int(y1 - y0)],
                    "iscrowd": 0
                }
                coco_output["annotations"].append(annotation_info)

                annotation_id += 1

                print(f"Processed {image_file} - Best mask score: {best_score:.3f}")
            except Exception as e:
                print(f"Failed to process {image_file}: {str(e)}")

# Save COCO annotations to a file
with open(os.path.join(annotations_save_dir, f"coco_panoptic_{path}.json"), 'w') as json_file:
    json.dump(coco_output, json_file, indent=4)

print("Processing complete.")


Processed 0a610d40-f5b8-4580-8e32-cc90b3620017___Com.G_TgS_FL 8191_flipTB.JPG - Best mask score: 1.015
Processed 0a2de4c5-d688-4f9d-9107-ace1d281c307___Com.G_TgS_FL 7941_180deg.JPG - Best mask score: 1.008
Processed 0a578317-a98b-4e26-b17b-cd2930330375___Com.G_TgS_FL 0785.JPG - Best mask score: 0.994
Processed Copy of 0a2de4c5-d688-4f9d-9107-ace1d281c307___Com.G_TgS_FL 7941_180deg.JPG - Best mask score: 1.008
Processing complete.


In [23]:
#=========================================================
#================== Verify Segmentation ==================
#=========================================================


import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

# Define paths
path = "train_test"
image_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/' + path
annotations_path = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/test/coco_panoptic_' + path + '.json'

# Load COCO annotations
coco = COCO(annotations_path)

# Helper function to display the segmentation mask and bounding box
def visualize_segmentation(image, segmentation, bbox):
    mask = np.array(segmentation, dtype=np.uint8)
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    color_mask[mask > 0] = [255, 0, 0]  # Red color for mask
    overlay = cv2.addWeighted(image, 1.0, color_mask, 0.5, 0)

    # Draw bounding box
    x, y, w, h = bbox
    cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green color for bbox

    return overlay

# Iterate over each image in the COCO annotations
for image_info in coco.loadImgs(coco.getImgIds()):
    image_id = image_info['id']
    image_filename = image_info['file_name']
    image_path = os.path.join(image_dir, os.path.relpath(image_filename, '/content/drive/MyDrive/plant_disease_detection/dataset'))

    # Load the image
    image = cv2.imread(image_filename)
    if image is None:
        print(f"Failed to load image {image_filename}")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load the annotations for the image
    annotation_ids = coco.getAnnIds(imgIds=image_id)
    annotations = coco.loadAnns(annotation_ids)

    # Overlay the segmentation masks and bounding boxes on the image
    overlay = image_rgb.copy()
    for ann in annotations:
        segmentation = ann['segmentation']
        bbox = ann['bbox']
        overlay = visualize_segmentation(overlay, segmentation, bbox)

    # Display the image with segmentation overlay and bounding boxes
    plt.figure(figsize=(6, 6))
    plt.imshow(overlay)
    plt.axis('off')
    plt.title(f"Image ID: {image_id} - {image_filename}")
    plt.show()


Output hidden; open in https://colab.research.google.com to view.